#### Importing all the necessary libraries



In [1]:
import numpy as np
import os
import pickle

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.layers import Dense,Flatten, Conv2D
from keras.layers import MaxPooling2D, Dropout
from keras.utils import np_utils, print_summary
import tensorflow as tf
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

Using TensorFlow backend.


####  Listing all files and directories present in the specified directory

In [0]:
dataset = os.listdir("/content/drive/My Drive/Artificial Intelligence/Data/")

In [3]:
dataset

['full_numpy_bitmap_apple.npy',
 'full_numpy_bitmap_bowtie.npy',
 'full_numpy_bitmap_candle.npy',
 'full_numpy_bitmap_door.npy',
 'full_numpy_bitmap_envelope.npy',
 'full_numpy_bitmap_fish.npy',
 'full_numpy_bitmap_guitar.npy',
 'full_numpy_bitmap_ice cream.npy',
 'full_numpy_bitmap_lightning.npy',
 'full_numpy_bitmap_moon.npy',
 'full_numpy_bitmap_mountain.npy',
 'full_numpy_bitmap_star.npy',
 'full_numpy_bitmap_tent.npy',
 'full_numpy_bitmap_toothbrush.npy',
 'full_numpy_bitmap_wristwatch.npy']

#### Creating an empty list to store the data

In [0]:
data=[]
feature = []
x = []
label = []

In [0]:
def load_data():
    count = 0
    for file in dataset:
        file = "/content/drive/My Drive/Artificial Intelligence/Data/" + file
        # Load arrays from .npy files
        data = np.load(file)
        # Casting the int numpy.ndarray to float numpy.ndarray
        # Dividing it by 255 helps to get the RGB color
        data = data.astype('float32') / 255.
        # Extracting top 10000 rows and all columns
        data = data[0:10000, :]
        # Storing the extracted data from each file as feature
        feature.append(data)
        x = [count for _ in range(10000)]
        count += 1
        x = np.array(x).astype('float32')
        # Reshaping the list to a single row 
        x = x.reshape(x.shape[0], 1)
        # Loading teh created data as label
        label.append(x)

    return feature, label

Storing the loaded feature and label of each file as features and labels

In [0]:
features, labels = load_data()

Casting the features and labels to float

In [0]:
features = np.array(features).astype('float32')
labels = np.array(labels).astype('float32')

#### Reshaping labels and features

In [0]:
features=features.reshape(features.shape[0]*features.shape[1],features.shape[2])
labels=labels.reshape(labels.shape[0]*labels.shape[1],labels.shape[2])

In [9]:
features.shape

(150000, 784)

In [10]:
labels.shape

(150000, 1)

#### To store the python objects as  byte stream

In [0]:
with open("features", "wb") as f:
    pickle.dump(features, f, protocol=4)
with open("labels", "wb") as f:
    pickle.dump(labels, f, protocol=4)

In [0]:
def keras_model(image_x, image_y):
    num_of_classes = 15
    model = Sequential()
    model.add(Conv2D(32, (5, 5), input_shape=(image_x,image_y,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(num_of_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    filepath = "QuickDraw.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]

    return model, callbacks_list

In [0]:
def loadFromPickle():
    with open("features", "rb") as f:
        features = np.array(pickle.load(f))
    with open("labels", "rb") as f:
        labels = np.array(pickle.load(f))

    return features, labels

In [0]:
def augmentData(features, labels):
    features = np.append(features, features[:, :, ::-1], axis=0)
    labels = np.append(labels, -labels, axis=0)
    return features, labels

In [0]:
def prepress_labels(labels):
    labels = np_utils.to_categorical(labels)
    return labels

In [0]:
features, labels = loadFromPickle()

In [0]:
features, labels = shuffle(features, labels)

In [0]:
labels=prepress_labels(labels)

In [0]:
train_x, test_x, train_y, test_y = train_test_split(features, labels, random_state=0,test_size=0.1)
train_x = train_x.reshape(train_x.shape[0], 28, 28, 1)
test_x = test_x.reshape(test_x.shape[0], 28, 28, 1)

In [20]:
model, callbacks_list = keras_model(28,28)
print_summary(model)






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 24, 24, 32)        832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 64)          51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800   

In [21]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=3, batch_size=64,
callbacks=[TensorBoard(log_dir="QuickDraw")])
model.save('QuickDraw.h5')

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 135000 samples, validate on 15000 samples







Epoch 1/3
135000/135000 [==============================] - 41s 301us/step - loss: 0.6424 - acc: 0.8158 - val_loss: 0.3003 - val_acc: 0.9144

Epoch 2/3
135000/135000 [==============================] - 40s 296us/step - loss: 0.3476 - acc: 0.9069 - val_loss: 0.2573 - val_acc: 0.9285
Epoch 3/3
135000/135000 [==============================] - 40s 299us/step - loss: 0.2927 - acc: 0.9223 - val_loss: 0.2485 - val_acc: 0.9328


In [0]:
score = model.evaluate(test_x, test_y, verbose=0)

In [23]:
score

[0.24854030679961045, 0.9328000000317891]

#### SVM Implementation

In [0]:
data=[]
feature = []
x = []
label = []
def load_data():
    count = 0
    for file in dataset:
        file = "/content/drive/My Drive/Artificial Intelligence/Data/" + file
        # Load arrays from .npy files
        data = np.load(file)
        # Casting the int numpy.ndarray to float numpy.ndarray
        # Dividing it by 255 helps to get the RGB color
        data = data.astype('float32') / 255.
        # Extracting top 10000 rows and all columns
        data = data[0:10000, :]
        # Storing the extracted data from each file as feature
        feature.append(data)
        x = [count for _ in range(10000)]
        count += 1
        x = np.array(x).astype('float32')
        # Reshaping the list to a single row 
        x = x.reshape(x.shape[0], 1)
        # Loading teh created data as label
        label.append(x)

    return feature, label

In [0]:
features, labels = load_data()

In [0]:
features = np.array(features).astype('float32')
labels = np.array(labels).astype('float32')

In [0]:
features=features.reshape(features.shape[0]*features.shape[1],features.shape[2])
labels=labels.reshape(labels.shape[0]*labels.shape[1],labels.shape[2])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, random_state=0,test_size=0.1)

In [0]:
clf = SVC(kernel='linear')

In [0]:
clf.fit(x_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
y_pred = clf.predict(x_test)
print(accuracy_score(y_test,y_pred))